### Get data from Steam Community Market API and save to csv

In [1]:
# Obtaining the data + inputting items seeking for

import re
import requests
import pandas as pd

dailyCookie = "76561199704981720%7C%7CeyAidHlwIjogIkpXVCIsICJhbGciOiAiRWREU0EiIH0.eyAiaXNzIjogInI6MTcwQl8yNDkzRTBDMF80QkU4NSIsICJzdWIiOiAiNzY1NjExOTk3MDQ5ODE3MjAiLCAiYXVkIjogWyAid2ViOmNvbW11bml0eSIgXSwgImV4cCI6IDE3MTk2MDgyNzQsICJuYmYiOiAxNzEwODgxODQ2LCAiaWF0IjogMTcxOTUyMTg0NiwgImp0aSI6ICIwRjJEXzI0QTRFNzFEX0Y0RDBGIiwgIm9hdCI6IDE3MTgzNjI3ODYsICJydF9leHAiOiAxNzM2NjE4ODA2LCAicGVyIjogMCwgImlwX3N1YmplY3QiOiAiODEuMTA1LjIwMS41NyIsICJpcF9jb25maXJtZXIiOiAiOTAuMTk3Ljc5LjEzMyIgfQ.g6KjGj3ELrBvIUy7EKa-kYl7b6nQcLRnuE-SVoCnASh129RqlPgDefOj75MUZGnR25UUUHMiYJ3FzjQqmSOcDQ"
items = ["Glove Case Key", "Officer Jacques Beltram | Gendarmerie Nationale", "Kilowatt Case", "AK-47 | Blue Laminate (Factory New)", "Glove Case"]

def fetch_item_to_df(item):
    # get historical price data of item from API
    url = "https://steamcommunity.com/market/pricehistory/"
    params = {
        'country': 'US',
        'currency': '1',
        'appid': '730',
        'market_hash_name': item
    }
    cookies = {'steamLoginSecure': dailyCookie}

    response = requests.get(url, params=params, cookies=cookies)
    jsonData = response.json()

    # print error message if request failed
    if response.status_code != 200:
        print(f"Failed to fetch data for {item}. Status code: {response.status_code}")
        return None

    # convert and clean data to dataframe object
    price_history = jsonData['prices']
    price_history_df = pd.DataFrame(price_history, columns=['date', 'price_usd', 'volume'])
    price_history_df['date'] = pd.to_datetime(price_history_df['date'].str[0:-4], format='%b %d %Y %H')
    price_history_df['volume'] = pd.to_numeric(price_history_df['volume'])
    price_history_df.set_index('date', inplace=True)
    grouped_current_item = price_history_df.groupby(pd.Grouper(freq='D')).agg({
    'price_usd':'median',
    'volume':'sum'
    })
    return grouped_current_item

def sanitize_filename(filename):
    """Sanitizes the filename to ensure it is valid for most operating systems."""
    filename = re.sub(r'[\\/*?:"<>|]', '_', filename)  # Replace disallowed characters with underscore
    filename = re.sub(r'\s+', '_', filename)  # Replace spaces with underscores
    return filename

def save_item_to_csv(item):
    csvData = './data/'+ sanitize_filename(item) +'.csv'
    fetch_item_to_df(item).to_csv(csvData, index=True)
    

#0     Nov 29 2016 01: +0      2.017   5261 - original format
## fetch and save data in items to csv - Uncomment this when checking new item or updating previous datasets

# for index, item in enumerate(items):
#     save_item_to_csv(item)






### Item Verification

In [2]:
# Call the method (From Get Data notebook) to get current_item dataframe

from matplotlib import pyplot as plt

current_item = fetch_item_to_df(items[4])

# shows what current item is, note: Array item stats with 0.
print(items[4])
print(current_item.tail())



Glove Case
            price_usd  volume
date                         
2024-06-23      5.613    2956
2024-06-24      5.648    2257
2024-06-25      5.647    3211
2024-06-26      5.672    2717
2024-06-27      5.750    3076


### Plotly start & end date

In [3]:
START_DATE = current_item.index[0]
END_DATE = current_item.index[-1]

In [31]:
import time

start_time = time.time()  # record start time

# Code to time
#df= fetch_item_to_df(items[4])
df = current_item

# moving average
df['smoothened_price'] = df['price_usd'].rolling(window=2).mean()

# candlestick 
new_data = df.resample('M').agg({
    'price_usd': ['max', 'min', 'mean'],
    'volume': 'sum'
})

# Slices the multi-level columns and renames
new_data.columns = ['high', 'low', 'mean', 'volume']

# exponential moving average
df['smoothened_price'] = df['price_usd'].ewm(span=2, adjust=False).mean()

# Bollinger bands
df['smoothened_price'] = df['price_usd'].rolling(window=20).mean()

# Calculate the rolling standard deviation for the 20-day window.
df['std_dev'] = df['price_usd'].rolling(window=20).std()

# Calculate the Bollinger Bands.
df['upper_band'] = df['smoothened_price'] + 2 * df['std_dev']
df['lower_band'] = df['smoothened_price'] - 2 * df['std_dev']


end_time = time.time()  # record end time
elapsed_time = end_time - start_time  # calculate elapsed time

print(f"The code took {elapsed_time} seconds to run.")


The code took 0.009661436080932617 seconds to run.


C:\Users\Nukul\AppData\Local\Temp\ipykernel_16536\1312727612.py:13: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



### Historic Price (0)  

In [4]:
# > Historic Price (Basic)

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item

# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
    # Filtering the data based on the date range
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['price_usd'], mode='lines+markers', name='Price USD'))

    # Formatting
    fig.update_layout(
        title='Historic Price over time',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True

        )

    fig.show()


# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)





interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph(start_date, end_date)>

### Volume Indicatior (1)

In [5]:
# > Volume Indicatior (1)

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item

# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
    # Filtering the data based on the date range
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Bar(x=filtered_data.index, y=filtered_data['volume'], name='Price USD'))

    # Formatting
    fig.update_layout(
        title='Volume Chart',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True
    )

    fig.show()


# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)



interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph(start_date, end_date)>

### Moving Averages (2)

In [6]:
# # SIMPLE Moving Average (SMA)                            
#  | Part of Moving Averages (2) block


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item

# calculates the 7-day moving average. 
# change window = to change period for average.
df['smoothened_price'] = df['price_usd'].rolling(window=2).mean()

# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
    # Filtering the data based on the date range
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['smoothened_price'], mode='lines', name='Smoothened Price'))

    # Formatting
    fig.update_layout(
        title='Simple Moving Average Chart',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True

        )

    fig.show()


# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)


interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph(start_date, end_date)>

In [7]:
 # SIMPLE Moving Average (SMA) -  CANDLESTICK  
 # | Part of Moving Averages (2) block

# NOTE: this produces candlesticks based on (specified time frame e.g. Weekly) data


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item

# Resample the data to weekly timeframe
# NOTE: Range can be altered: 'W' (week), 'M' (Month), '2D' (2 Days)
new_data = df.resample('M').agg({
    'price_usd': ['max', 'min', 'mean'],
    'volume': 'sum'
})


# Slices the multi-level columns and renames
new_data.columns = ['high', 'low', 'mean', 'volume']
# print(new_data.head(10))

# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):


    mask = (new_data.index >= pd.to_datetime(start_date)) & (new_data.index <= pd.to_datetime(end_date))
    filtered_data = new_data.loc[mask]

    
    # Creating the candlestick chart
    fig = go.Figure(data=[go.Candlestick(x=filtered_data.index,
                                         open=filtered_data['mean'],
                                         high=filtered_data['high'],
                                         low=filtered_data['low'],
                                         close=filtered_data['mean'],
                                         name='Candlestick')])
    


    # Formatting
    fig.update_layout(
        title='SMA - Candlestick',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        
        showlegend=True
    )

    fig.show()


# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)


C:\Users\Nukul\AppData\Local\Temp\ipykernel_16536\2351391807.py:17: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph(start_date, end_date)>

### Exponential Moving Average

In [8]:
 # Exponential Moving Average (EMA)  
 # | Part of Moving Averages (2) block

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item

# change span = to change smoothing factor/span for average.
df['smoothened_price'] = df['price_usd'].ewm(span=2, adjust=False).mean()
# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
    # Filtering the data based on the date range
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['smoothened_price'], mode='lines', name='Smoothened Price'))

    # Formatting
    fig.update_layout(
        title='Exponential Moving Average Chart',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True

        )
    
    fig.show()

# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph(start_date, end_date)>

### Regression modelling

In [9]:
# Linear Regression (3.1)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

current_item['date_ordinal'] = current_item.index.map(pd.Timestamp.toordinal)
X = current_item['date_ordinal'].values.reshape(-1, 1)
y = current_item['price_usd'].values


# train test split (first 80% train, last 20% test)
split_point = int(len(X) * 0.80)  
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]


# intialize the model (like initialising a variable)
model_lr = LinearRegression()
model_lr.fit(X, y)

# make predictions
y_pred = model_lr.predict(X)


def update_graph(start_date, end_date):
    # Filtering the data based on the date range
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]
    filtered_pred = y_pred[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_data.index[:split_point], y=filtered_data['price_usd'], mode='lines+markers', name='Price USD'))

     # Add the test set prices
    fig.add_trace(go.Scatter(x=filtered_data.index[split_point:], y=y_test, mode='lines+markers', name='Test Set Data', line=dict(dash='dash')))


    # Add the predicted prices
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_pred, mode='lines', name='Regression Line', line=dict(dash='dot')))

    # Formatting
    fig.update_layout(
        title='Linear Regression ML Model (3.1)',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True

        )

    fig.show()


# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)




interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph(start_date, end_date)>

In [10]:
# Linear Regression (3.1) FUTURE MODELLING STRATEGY

# Linear Regression (3.1)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

current_item['date_ordinal'] = current_item.index.map(pd.Timestamp.toordinal)
X = current_item['date_ordinal'].values.reshape(-1, 1)
y = current_item['price_usd'].values

# THIS SECTION IS FOR FUTURE PRICE PREDICTION:
future_dates = pd.date_range(start=current_item.index[-1], end='2025-12-31')
future_dates_ordinal = future_dates.map(pd.Timestamp.toordinal).values.reshape(-1, 1)

# intialize the model (like initialising a variable)
model_lr = LinearRegression()
model_lr.fit(X, y)

# make predictions
y_pred = model_lr.predict(future_dates_ordinal)

# Create a DataFrame for future predictions
future_df = pd.DataFrame(data={'date': future_dates, 'price_usd': y_pred})
future_df.set_index('date', inplace=True)

# Combine current_item and future_df
combined_df = pd.concat([current_item, future_df])

def update_graph(start_date, end_date):
    # Filtering the data based on the date range
    mask = (combined_df.index >= pd.to_datetime(start_date)) & (combined_df.index <= pd.to_datetime(end_date))
    filtered_data = combined_df.loc[mask]

    # Creating the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['price_usd'], mode='lines+markers', name='Price USD'))

    # Add the predicted prices
    fig.add_trace(go.Scatter(x=future_df.index, y=future_df['price_usd'], mode='lines', name='Predicted Price', line=dict(dash='dot')))

    # Formatting
    fig.update_layout(
        title='Linear Regression ML Model (3.1)',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(  # Added lines for range slider
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True
    )

    fig.show()

# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=combined_df.index[0], disabled=False)
end_date_picker = DatePicker(description='End Date', value=combined_df.index[-1], disabled=False)

# Display the interactive widget
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)


interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph(start_date, end_date)>

Bollinger Bands

In [11]:
# # Bollinger Bands - Bolligner Bands currently use the SMA but the chart depicts the historical price                           


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# Assuming 'current_item' is your DataFrame with a DateTime index and a 'price_usd' column.
df = current_item

# Calculate the 20-day moving average. Change window=20 to change the period for the average.
df['smoothened_price'] = df['price_usd'].rolling(window=20).mean()

# Calculate the rolling standard deviation for the 20-day window.
df['std_dev'] = df['price_usd'].rolling(window=20).std()

# Calculate the Bollinger Bands.
df['upper_band'] = df['smoothened_price'] + 2 * df['std_dev']
df['lower_band'] = df['smoothened_price'] - 2 * df['std_dev']

# Function to update the graph based on the selected date range.
def update_graph(start_date, end_date):
    # Filtering the data based on the date range.
    mask = (df.index >= pd.to_datetime(start_date)) & (df.index <= pd.to_datetime(end_date))
    filtered_data = df.loc[mask]

    # Creating the plot.
    fig = go.Figure()

    # Adding the Historical Price line.
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['price_usd'], mode='lines', name='Historical Price'))

    # Adding the upper Bollinger Band line.
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['upper_band'], mode='lines', name='Upper Bollinger Band', line=dict(color='red', dash='dash')))

    # Adding the lower Bollinger Band line.
    fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['lower_band'], mode='lines', name='Lower Bollinger Band', line=dict(color='blue', dash='dash')))

    # Formatting the plot.
    fig.update_layout(
        title='Historical Price and Bollinger Bands Chart',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        xaxis=dict(
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
        showlegend=True
    )

    fig.show()

# Creating interactive widgets for date selection.
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget.
interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date'), DatePicker…

<function __main__.update_graph(start_date, end_date)>